# 일관성 점수 코드 시도

In [1]:
import pandas as pd

data = {
    "문장": [
        "She go to the store yesterday.",
        "They enjoys watching movies.",
        "This is a perfect example of the method.",
        "He walk to school every day.",
        "The cat is chasing it tail."
    ],
    "포함된 단어": ["go", "movies", "example", "walk", "cat"]
}

df = pd.DataFrame(data)
df.to_csv("test_data.csv", index=False, encoding='cp949')


In [6]:
!pip install --upgrade transformers

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 10.0/10.0 MB 88.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.2
    Uninstalling transformers-4.45.2:
      Successfully uninstalled transformers-4.45.2


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# 모델과 토크나이저 로드
model_name = "Qwen/Qwen2-VL-7B-Instruct"  # 모델 이름
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


# 의미 일관성 점수 측정 함수
def measure_semantic_coherence(sentence):
    prompt = f"Rate the semantic coherence of the following sentence on a scale from 0 to 1:\n\n'{sentence}'\n\nScore:"
    inputs = tokenizer(prompt, return_tensors="pt")

    # 텍스트 인코딩 및 점수 계산 예시
    with torch.no_grad():
        outputs = model(**inputs)

    # 의미 일관성 점수 (예: 출력 벡터의 평균값을 사용)
    coherence_score = float(outputs.last_hidden_state.mean().item())
    return coherence_score


# 예시 문장
sentences = [
    "The cat is sleeping on the bed.",
    "She go to the store yesterday.",
    "asdfasdfasdfasdfasdfasdfasdf",
]

# 각 문장에 대해 의미 일관성 점수 계산
for sentence in sentences:
    score = measure_semantic_coherence(sentence)
    print(f"문장: {sentence}\n의미 일관성 점수: {score}\n")

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


: 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# FLAN-T5 모델 로드
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# 의미 일관성 및 문법 평가 함수
def measure_semantic_coherence(sentence):
    # 구체적인 평가 프롬프트 생성
    prompt = (
        f"Evaluate the following sentence for both grammatical correctness and semantic coherence. "
        f"Answer only '1' if the sentence is both grammatically correct and semantically coherent. "
        f"Answer only '0' if the sentence has grammatical errors or lacks coherence.\n\n"
        f"Sentence: '{sentence}'\n\nAnswer with '1' or '0' only:\n\nScore:"
    )

    # 모델 입력 준비
    inputs = tokenizer(prompt, return_tensors="pt")

    # 모델 예측 생성
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=20)  # max_length를 20으로 늘림

    # 예측에서 점수 추출
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Model Response: {response_text}")  # 디버깅을 위해 모델 응답 출력
    try:
        # 응답에서 숫자만 추출
        coherence_score = float(response_text.strip())
    except ValueError:
        coherence_score = None  # 잘못된 출력일 경우 처리

    return coherence_score


# 의미 일관성이 높은 문장과 낮은 문장 예시 (실제 라벨 포함)
data = [
    # (문장, 실제 라벨)
    ("The dog barks loudly at night.", 1),
    ("She went to the store to buy some milk.", 1),
    ("The sun is very bright today.", 1),
    ("He reads a book every evening before bed.", 1),
    ("The students are working on their project together.", 1),
    ("The apple sings softly to the wind.", 0),
    ("She quickly swims through the library shelves.", 0),
    ("The car danced across the road.", 0),
    ("They painted the silence with laughter.", 0),
    ("The sky ate all the music at sunset.", 0),
]

# 예측과 실제 라벨 비교
correct_predictions = 0
total_predictions = len(data)

for sentence, true_label in data:
    predicted_score = measure_semantic_coherence(sentence)
    # 예측 점수를 이진 라벨로 변환 (1.0이면 1, 그 외는 0)
    predicted_label = 1 if predicted_score == 1.0 else 0

    print(f"Sentence: {sentence}")
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}\n")

    # 정확도 계산
    if predicted_label == true_label:
        correct_predictions += 1

accuracy = correct_predictions / total_predictions
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Response: 1
Sentence: The dog barks loudly at night.
True Label: 1, Predicted Label: 1

Model Response: 1
Sentence: She went to the store to buy some milk.
True Label: 1, Predicted Label: 1

Model Response: 1
Sentence: The sun is very bright today.
True Label: 1, Predicted Label: 1

Model Response: 1
Sentence: He reads a book every evening before bed.
True Label: 1, Predicted Label: 1

Model Response: 1
Sentence: The students are working on their project together.
True Label: 1, Predicted Label: 1

Model Response: 1
Sentence: The apple sings softly to the wind.
True Label: 0, Predicted Label: 1

Model Response: 0
Sentence: She quickly swims through the library shelves.
True Label: 0, Predicted Label: 0

Model Response: 1
Sentence: The car danced across the road.
True Label: 0, Predicted Label: 1

Model Response: 0
Sentence: They painted the silence with laughter.
True Label: 0, Predicted Label: 0

Model Response: 0
Sentence: The sky ate all the music at sunset.
True Label: 0, Pre